I am going to use `gemini-1.5-flash-8b` using google api key. One of the best features of Google's Gemini API is its intuitive chat handling. Unlike other LLM APIs, you don't need to specify roles like "user" or "assistant" - Gemini handles this automatically. Other APIs, such as OpenAI, often require something like:
```python
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hello"},
    {"role": "assistant", "content": "Hi there!"},
    {"role": "user", "content": "How are you?"}
]
```

We can also utilize the libraries like `gpt4all`  to load LLM modules via the Python SDK and API (without downloading locally, though local usage is supported).

Reference: [GPT4All Documentation](https://docs.gpt4all.io/index.html)

Install `gpt4all`: pip install gpt4all

Other frameworks to run LLM locally: [Frameworks Guide](https://www.datacamp.com/tutorial/run-llms-locally-tutorial)

In [2]:
# import required libraries and modules
import os
from dotenv import load_dotenv         # pip install python-dotenv
import google.generativeai as genai    # pip install google-generativeai

In [ ]:
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
office_name = 'Raj dental clinic'
system_prompt = f"""You are a dental office AI assistant specializing in insurance verification calls from {office_name}.
Your role is to:
1. Verify patient insurance coverage and benefits
2. Gather information about deductibles, maximums, and coverage percentages
3. Handle insurance-related inquiries professionally and concisely
4. Respond in a conversational yet professional manner

Start the call with a professional introduction like: 
"Hi, this is [Assistant Name] calling from {office_name}. I'm calling to verify insurance benefits for our patient [Patient Name]."
"""
genai.configure(api_key=api_key)
        
model = genai.GenerativeModel(model_name = 'gemini-1.5-flash-8b')      
chat = model.start_chat(history = [])
start_conversation = chat.send_message(system_prompt).text

# convert above text to speech then audio
import base64
from io import BytesIO
import numpy as np
import scipy.io
from TTS.api import TTS
import simpleaudio as sa
import logging, warnings
from scipy.io import wavfile
import torch


# Suppress warnings and verbose output
warnings.filterwarnings("ignore")
logging.getLogger("TTS.utils.io").setLevel(logging.ERROR)
logging.getLogger("TTS.utils.audio").setLevel(logging.ERROR)
logging.getLogger("TTS").setLevel(logging.ERROR)

# Initialize TTS with optimization
device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS(
    model_name="tts_models/en/ljspeech/tacotron2-DDC",
    progress_bar = False
).to(device)

# Convert text to speech and save to file
# tts.tts_to_file(text = start_conversation, speed = 2.0, file_path = 'output.wav')

# encoding function
# def encode_wav(wav):
#     wav = np.array(wav)
#     wav_norm = wav * (32767 / max(0.01, np.max(np.abs(wav))))
#     wav_norm = wav_norm.astype(np.int16)
#     wav_obj = sa.WaveObject(wav_norm, 1, 2, 22050)
#     return wav_obj


# Generate speech
wav = tts.tts(text = start_conversation, speed = 1.2)

# Convert to numpy array and normalize
wav = np.array(wav)
wav_norm = wav * (32767 / max(0.01, np.max(np.abs(wav))))
wav_norm = wav_norm.astype(np.int16)

# Create and play audio object
wav_obj = sa.WaveObject(wav_norm, 1, 2, 22050)
play_obj = wav_obj.play()
play_obj.wait_done()

 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

# Speech to Text Conversion Using Whisper and SpeechRecognition

This code outlines the steps to convert spoken language into text using Python libraries: `SpeechRecognition` and `Whisper`.

#### Libraries to Install

```bash
pip install SpeechRecognition
pip install openai-whisper
pip install pyaudio  # Required for microphone input with SpeechRecognition


In [66]:
# Speak into mic and convert it to text
# pip install SpeechRecognition
# pip install openai-whisper
# pip install pyaudio  # Required for microphone input with speech_recognition
# Initialize recognizer and Whisper model
import speech_recognition as sr
import whisper
recognizer = sr.Recognizer()
model = whisper.load_model("base")  # or "tiny", "small", "medium", "large"
# Use microphone as source
with sr.Microphone() as source:
    print("Adjusting for ambient noise... Please wait...")
    recognizer.adjust_for_ambient_noise(source, duration = 1)
    
    print("\nListening... Speak now!")
    audio = recognizer.listen(source)
    
    print("Processing speech with Whisper...")
    try:
        # Convert speech to text using Whisper
        text = recognizer.recognize_whisper(audio, model = "base")
        print("You said:", text)
    except sr.UnknownValueError:
        print("Whisper could not understand audio")
    except Exception as e:
        print("Error during recognition:", str(e))

Adjusting for ambient noise... Please wait...

Listening... Speak now!
Processing speech with Whisper...
You said:  Hi, what's up?


## How Gemini GenerativeModel works?

The `genai.GenerativeModel` class in the `google.generativeai` library wraps default parameters for calls to `generate_content`, `count_tokens`, and `start_chat`. This class supports multi-turn conversations and multimodal requests. The supported media types for input and output depend on the model.

#### Usage

#### Importing and Configuration
First, import the necessary libraries and configure the API key:
```python
import google.generativeai as genai
import PIL.Image
genai.configure(api_key='YOUR_API_KEY')


In [41]:
model = genai.GenerativeModel('gemini-1.5-flash-8b')
model?

Type:        GenerativeModel
String form:
genai.GenerativeModel(
    model_name='models/gemini-1.5-flash-8b',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)
File:        c:\users\anita\desktop\godental_ai\myenv\lib\site-packages\google\generativeai\generative_models.py
Docstring:  
The `genai.GenerativeModel` class wraps default parameters for calls to
`GenerativeModel.generate_content`, `GenerativeModel.count_tokens`, and
`GenerativeModel.start_chat`.

This family of functionality is designed to support multi-turn conversations, and multimodal
requests. What media-types are supported for input and output is model-dependant.

>>> import google.generativeai as genai
>>> import PIL.Image
>>> genai.configure(api_key='YOUR_API_KEY')
>>> model = genai.GenerativeModel('models/gemini-pro')
>>> result = model.generate_content('Tell me a story about a magic backpack')
>>> result.text
"In the quaint little town of Lakeside

### Gemini Chat History Format: Accessing Response and Role

```python
chat_history = [
   {
       "parts": [{"text": "You are a dental assistant AI helping with insurance verification. Be professional and concise."}],
       "role": "user"
   },
   {
       "parts": [{"text": "Okay, I'm ready. Please provide the patient's name, date of birth, insurance information (policy number, group number if applicable), and the date of service."}],
       "role": "model"
   },
   {
       "parts": [{"text": "Can you check insurance coverage for a patient Matthew?"}],
       "role": "user"
   }
]

# To access messages:
for message in chat_history:
   print(f"{message['role']}: {message['parts'][0]['text']}\n")

# Get specific messages:
first_user_message = chat_history[0]['parts'][0]['text']
last_model_response = chat_history[-1]['parts'][0]['text']

# Get all user messages:
user_messages = [
   msg['parts'][0]['text'] 
   for msg in chat_history 
   if msg['role'] == "user"
]

## Text-to-Speech (TTS) with Coqui TTS Library

Text-to-Speech (TTS) is a technology that converts text into spoken audio. The Coqui TTS library provides multiple pre-trained models for high-quality speech synthesis.

### Installation
Follow this link for [TTS installation](https://www.youtube.com/watch?v=zRaDe08cUIk&list=PL19C7uchWZerUT0qIiEv7m2zXBs5kYl1L&index=12).
It may require some dependencies to install for Winodows operating system.
```markdown
pip install TTS
```

### Available Models
```markdown
1. Fast but lower quality
tts = TTS(model_name="tts_models/en/ljspeech/fast_pitch")

2. Balanced speed/quality
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC_ph")

3. Best quality but slower
tts = TTS(model_name="tts_models/en/ljspeech/glow-tts")

4. Support multiple languages
tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2")
```

### Understanding Model Name
```markdown
Format: tts_models/<language>/<dataset>/<model_architecture>
Example: tts_models/en/ljspeech/tacotron2-DDC_ph
```

Components:
- tts_models: Type of model
- en: Language code
- ljspeech: Training dataset
- tacotron2-DDC_ph: Model architecture

In [3]:
# Use Hugging Face
from TTS.api import TTS
tts = TTS("tts_models/en/ljspeech/tacotron2-DDC")

 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024


c:\Users\anita\Desktop\godental_ai\myenv\lib\site-packages\TTS\utils\io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, 

 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Generator Model: hifigan_generator
 > Discriminator Model: hifigan_discriminator
Removing weight norm...


## Text Embedding and Similarity Search Using FAISS and HuggingFace

### Overview
We are creating a text embedding and similarity search system using the FAISS vector store and HuggingFace embeddings. This system converts text into numerical vectors and performs similarity searches to find the most relevant documents.

### How It Works
1. **Text Embedding**: Text is converted into numerical vectors using HuggingFace embeddings.
2. **Vector Store**: FAISS creates a vector store from these embeddings.
3. **Similarity Search**: Searches the vector store for the most similar vectors to a query vector.

### FAISS
FAISS (Facebook AI Similarity Search) is a library developed by Meta's Fundamental AI Research (FAIR) team for efficient similarity search and clustering of dense vectors . It supports various algorithms and can handle large datasets efficiently [FAISS](https://github.com/facebookresearch/faiss).

### Semantic Search
Semantic search is a search technique that understands the meaning and context of queries, rather than just matching keywords. [What is semantic search, and how does it work?](https://cloud.google.com/discover/what-is-semantic-search). It uses technologies like machine learning and vector search to deliver more relevant results based on the searcher's intent and context. [A Comprehensive Semantic Search Guide - Elastic](https://www.elastic.co/what-is/semantic-search).



In [ ]:
import warnings
warnings.filterwarnings('ignore')
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS

def faiss_score(chunked_text):
    # Initialize the embeddings model with specific arguments for huggingface_hub 0.25.0
    embeddings = HuggingFaceInstructEmbeddings(
        model_name="hkunlp/instructor-base",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
        cache_folder=None
    )
    
    # Create the FAISS vector store
    faiss_index = FAISS.from_texts(
        texts = chunked_text,
        embedding = embeddings
    )
    
    return faiss_index

In [10]:
# Example 1: Create a simple list of insurance terms that might be misheard
simple_texts = ["insurance", "policy", "claim", "health insurance", "life insurance"]

# Convert the text list into FAISS vector store for similarity searching
# This process converts each text into a numerical vector representation
vector_store = faiss_score(simple_texts)

# View all documents in the index by performing an empty search ("")
# k=len(simple_texts) ensures we get back all documents
print("Example 1 - Simple insurance terms:")
docs = vector_store.similarity_search("", k = len(simple_texts))
# Enumerate through results and print each document with its index
for i, doc in enumerate(docs):
   print(f"Document {i}: {doc.page_content}")

# Example 2: Create a more realistic list of insurance terms that might be misheard
insurance_texts = [
   "health insurance",
   "life insurance",
   "policy number",
   "claim status",
   "deductible"
]
# Create another vector store with these insurance terms
vector_store2 = faiss_score(insurance_texts)

# View all documents in the second vector store
print("\nExample 2 - Insurance terms:")
docs2 = vector_store2.similarity_search("", k=len(insurance_texts))
for i, doc in enumerate(docs2):
   print(f"Document {i}: {doc.page_content}")

# Example 3: Demonstrate similarity search functionality
# Search for terms similar to "insurance", limiting results to top 2 matches (k=2)
print("\nExample 3 - Similarity search for 'insurance':")
results = vector_store2.similarity_search("insurance", k = 2)
# Print the most similar terms found
for i, doc in enumerate(results):
   print(f"Similar document {i}: {doc.page_content}")

# Example 4: Demonstrate with misheard variations
misheard_insurance = [
   "health insurance",
   "helt insurence",  # misheard version
   "life insurance", 
   "date of worth",  # misheard version
   "data box",
   "polisy number"    # misheard version
]

# Create vector store for misheard terms
vector_store3 = faiss_score(misheard_insurance)

# Search for similar phrases
print("\nExample 4 - Searching for 'date of birth' with misheard variations:")
results = vector_store3.similarity_search("date of birth", k=3)
for i, doc in enumerate(results):
   print(f"Match {i}: {doc.page_content}")

load INSTRUCTOR_Transformer
max_seq_length  512
Example 1 - Simple insurance terms:
Document 0: policy
Document 1: claim
Document 2: insurance
Document 3: health insurance
Document 4: life insurance
load INSTRUCTOR_Transformer
max_seq_length  512

Example 2 - Insurance terms:
Document 0: deductible
Document 1: claim status
Document 2: policy number
Document 3: health insurance
Document 4: life insurance

Example 3 - Similarity search for 'insurance':
Similar document 0: health insurance
Similar document 1: life insurance
load INSTRUCTOR_Transformer
max_seq_length  512

Example 4 - Searching for 'date of birth' with misheard variations:
Match 0: date of worth
Match 1: data box
Match 2: polisy number


In [24]:
import pandas as pd
# Read the CSV file
df = pd.read_csv("./correction_lookup.csv")

# Create FAISS index for misheard terms
misheard_texts = df['misheard'].tolist()
faiss_index = faiss_score(misheard_texts)

# Create a function to store indices in the dataframe
def add_faiss_indices(df, faiss_index):
    # Get all documents with their indices using an empty search
    docs = faiss_index.similarity_search("", k = len(df))
    
    # Create a mapping of document content to index
    doc_to_index = {doc.page_content: i for i, doc in enumerate(docs)}
    
    # Add indices to dataframe
    df['faiss_index'] = df['misheard'].map(doc_to_index)
    
    return df

# Update dataframe with FAISS indices
df = add_faiss_indices(df, faiss_index)

# Save updated dataframe
df.to_csv("correction_lookup_with_faiss.csv", index = False)

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
def find_similar_terms(query, faiss_index, df, k=5, threshold=0.5):
    # Search for similar terms with scores
    similar_docs_with_scores = faiss_index.similarity_search_with_score(query, k=k)
    
    # Get the corrections and confidence scores
    results = []
    for doc, score in similar_docs_with_scores:
        # Convert distance score to confidence (0-1 range)
        confidence = 1 / (1 + score)
        
        # Only include matches above threshold
        if confidence >= threshold:
            misheard = doc.page_content
            correction = df[df['misheard'] == misheard]['correction'].iloc[0]
            results.append({
                'misheard': misheard,
                'correction': correction, 
                'confidence': confidence
            })
    
    # Sort results by confidence score in descending order
    results = sorted(results, key=lambda x: x['confidence'], reverse=True)
    
    if results:
        best_match = results[0]  # Get the highest confidence match
        print(f"\nBest match: {best_match['correction']} (confidence: {best_match['confidence']:.2f})")
        
        print("\nDid you mean:")
        for result in results:
            print(f"- {result['correction']} (misheard as: {result['misheard']}, confidence: {result['confidence']:.2f})")
            
        return results  # Return the full list of results
    else:
        print("No matches found above threshold")
        return []  # Return empty list if no matches

# Example usage:
test_query = "memory id"
similar_terms = find_similar_terms(test_query, faiss_index, df, k = 5, threshold = 0.5)


Best match: member id (confidence: 0.99)

Did you mean:
- member id (misheard as: memory id, confidence: 0.99)
- member id (misheard as: mem-er id, confidence: 0.91)
- member id (misheard as: memo id, confidence: 0.88)
- member id (misheard as: member i'd, confidence: 0.87)
- member id (misheard as: member id, confidence: 0.86)


# Testing Extraction Function Using LLM

The code below shows the process of testing an extraction function using a large language model (LLM) for insurance verification.

## Setup and Initialization

1. **Prepare the Environment:**
   - Load necessary libraries for environment management, data handling, and API interactions.
   - Set up environment variables and configure the Generative Model using an API key.

2. **Initialize the Class:**
   - Define a class named `InsuranceVerifier` to handle insurance verification tasks.
   - Set up the system prompts to guide the AI assistant's role and interactions.

## Extraction Functions

Each function is designed to extract specific information from text using prompts sent to the LLM:

1. **Extract Insurance Status:**
   - This function identifies whether the patient’s insurance status is active or inactive.
   - The model processes the text and returns "Active," "Inactive," or "None" if unclear.

2. **Extract Dates:**
   - This function extracts dates from text and converts them into a standard format (MM/DD/YYYY).
   - It validates the extracted date to ensure it is correctly formatted.

3. **Extract Monetary Amounts:**
   - This function identifies dollar amounts mentioned in the text.
   - It returns the amount as a numeric value or "None" if no amount is found.

4. **Extract Percentages:**
   - This function extracts percentage values from the text.
   - It ensures the extracted value is within a valid range (0-100).

5. **Extract Plan Types:**
   - This function identifies the type of insurance plan (e.g., PPO, HMO) from the text.
   - It returns the plan type or "None" if the type is unclear.

6. **Extract Group Numbers:**
   - This function extracts the insurance group number from the text.
   - It returns the group number or "None" if not found.

7. **Extract Time Periods:**
   - This function identifies time periods such as benefit periods or waiting periods from the text.
   - It returns the period in a common format (e.g., "Calendar Year," "6 months").

8. **Extract Frequency Limitations:**
   - This function extracts frequency limitations, such as how often a benefit can be used.
   - It returns the frequency or "None" if not found.

9. **Extract Boolean Values:**
   - This function identifies yes/no or required/not required answers from the text.
   - It returns `True`, `False`, or "None" if unclear.

## Testing

1. **Create an Instance:**
   - Instantiate the `InsuranceVerifier` class with the relevant office name.

2. **Use Extraction Functions:**
   - Apply the extraction functions to process text and retrieve information.

3. **Handle Extracted Information:**
   - Display or handle the extracted information as needed for verification purposes.

This setup leverages the power of LLM to improve the efficiency and accuracy of extracting insurance-related information in a professional context.


In [32]:
import os
from typing import Optional, Dict, Any
from dataclasses import dataclass
from datetime import datetime
import re
from dotenv import load_dotenv
import google.generativeai as genai
import random

class InsuranceVerifier:
    def __init__(self, office_name: str):
        # Load environment variables and setup
        load_dotenv()
        api_key = os.getenv('GOOGLE_API_KEY')
        self.office_name = office_name
        
        # Configure Gemini
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name='gemini-1.5-flash-8b')
        
        # Initialize chat with system prompt
        self.system_prompt = f"""You are a dental office AI assistant specializing in insurance verification calls from {office_name}.
        Your role is to:
        1. Verify patient insurance coverage and benefits
        2. Gather information about deductibles, maximums, and coverage percentages
        3. Handle insurance-related inquiries professionally and concisely
        4. Respond in a conversational yet professional manner

        Start the call with a professional introduction like: 
        "Hi, this is [Assistant Name] calling from {office_name}. I'm calling to verify insurance benefits for our patient [Patient Name]."
        """
        
        self.chat = self.model.start_chat(history=[])
        self.start_conversation = self.chat.send_message(self.system_prompt).text
        
        # Setup extraction helpers
        self.extract_helpers = {
            'eligibility': {
                'status': self.extract_status,
                'effective_date': self.extract_date,
                'plan_type': self.extract_plan_type,
                'group_number': self.extract_group_number
            },
            'benefits': {
                'annual_maximum': self.extract_amount,
                'remaining_maximum': self.extract_amount,
                'deductible': self.extract_amount,
                'deductible_met': self.extract_amount,
                'benefit_period': self.extract_period
            },
            'coverage': {
                'preventive': self.extract_percentage,
                'basic': self.extract_percentage,
                'major': self.extract_percentage,
                'periodontics': self.extract_percentage,
                'endodontics': self.extract_percentage
            },
            'limitations': {
                'waiting_period': self.extract_period,
                'frequency': self.extract_frequency,
                'missing_tooth': self.extract_boolean,
                'pre_authorization': self.extract_boolean
            }
        }

    def extract_status(self, text: str) -> Optional[str]:
        """Extract insurance status from text."""
        try:
            prompt = f"""Extract the insurance status from the following text.
            Return only 'Active' if the patient is eligible/active, 'Inactive' if not eligible,
            or 'None' if unclear. No other text.

            Text: {text}
            """
            
            response = self.chat.send_message(prompt)
            result = response.text.strip()
            
            if result in ['Active', 'Inactive']:
                return result
            return None
            
        except Exception as e:
            print(f"Error extracting status: {str(e)}")
            return None

    def extract_date(self, text: str) -> Optional[str]:
        """Extract date in MM/DD/YYYY format."""
        try:
            prompt = f"""Extract the date from the following text and convert it to MM/DD/YYYY format.
            If no valid date is found, respond with 'None'.
            Only return the formatted date or 'None', no other text.

            Text: {text}
            """
            
            response = self.chat.send_message(prompt)
            result = response.text.strip()
            
            if result == 'None':
                return None
                
            # Validate the response looks like a date
            parts = result.split('/')
            if len(parts) == 3 and all(p.isdigit() for p in parts):
                return result
            
            return None
            
        except Exception as e:
            print(f"Error extracting date: {str(e)}")
            return None

    def extract_amount(self, text: str) -> Optional[float]:
        """Extract monetary amount from text."""
        try:
            prompt = f"""Extract the dollar amount from the following text.
            Return only the number (no $ or commas) or 'None' if no amount found.
            
            Text: {text}
            """
            
            response = self.chat.send_message(prompt)
            result = response.text.strip()
            
            if result == 'None':
                return None
                
            try:
                return float(result)
            except ValueError:
                return None
                
        except Exception as e:
            print(f"Error extracting amount: {str(e)}")
            return None

    def extract_percentage(self, text: str) -> Optional[int]:
        """Extract percentage from text."""
        try:
            prompt = f"""Extract the percentage from the following text.
            Return only the number (no % symbol) or 'None' if no percentage found.
            
            Text: {text}
            """
            
            response = self.chat.send_message(prompt)
            result = response.text.strip()
            
            if result == 'None':
                return None
                
            try:
                percentage = int(result)
                if 0 <= percentage <= 100:
                    return percentage
                return None
            except ValueError:
                return None
                
        except Exception as e:
            print(f"Error extracting percentage: {str(e)}")
            return None

    def extract_plan_type(self, text: str) -> Optional[str]:
        """Extract insurance plan type from text."""
        try:
            prompt = f"""Extract the insurance plan type from the following text.
            Common types include: PPO, HMO, DHMO, Indemnity, etc.
            Return only the plan type or 'None' if unclear. No other text.
            
            Text: {text}
            """
            
            response = self.chat.send_message(prompt)
            result = response.text.strip()
            
            if result == 'None':
                return None
            return result
            
        except Exception as e:
            print(f"Error extracting plan type: {str(e)}")
            return None

    def extract_group_number(self, text: str) -> Optional[str]:
        """Extract group number from text."""
        try:
            prompt = f"""Extract the insurance group number from the following text.
            Return only the group number or 'None' if not found. No other text.
            The group number might be labeled as 'Group #', 'Group Number', or similar.
            
            Text: {text}
            """
            
            response = self.chat.send_message(prompt)
            result = response.text.strip()
            
            if result == 'None':
                return None
            return result
            
        except Exception as e:
            print(f"Error extracting group number: {str(e)}")
            return None

    def extract_period(self, text: str) -> Optional[str]:
        """Extract benefit or waiting period from text."""
        try:
            prompt = f"""Extract the time period from the following text.
            Common formats include: 'Calendar Year', 'Contract Year', '6 months', '12 months', etc.
            Return only the period or 'None' if not found. No other text.
            
            Text: {text}
            """
            
            response = self.chat.send_message(prompt)
            result = response.text.strip()
            
            if result == 'None':
                return None
            return result
            
        except Exception as e:
            print(f"Error extracting period: {str(e)}")
            return None

    def extract_frequency(self, text: str) -> Optional[str]:
        """Extract frequency limitations from text."""
        try:
            prompt = f"""Extract the frequency limitation from the following text.
            Common formats include: 'Once every 6 months', '2 per year', 'Annual', etc.
            Return only the frequency or 'None' if not found. No other text.
            
            Text: {text}
            """
            
            response = self.chat.send_message(prompt)
            result = response.text.strip()
            
            if result == 'None':
                return None
            return result
            
        except Exception as e:
            print(f"Error extracting frequency: {str(e)}")
            return None

    def extract_boolean(self, text: str) -> Optional[bool]:
        """Extract yes/no answer from text."""
        try:
            prompt = f"""Is the following text indicating 'yes'/'required' or 'no'/'not required'?
            Return only 'True', 'False', or 'None' if unclear. No other text.
            
            Text: {text}
            """
            
            response = self.chat.send_message(prompt)
            result = response.text.strip().lower()
            
            if result == 'true':
                return True
            elif result == 'false':
                return False
            return None
            
        except Exception as e:
            print(f"Error extracting boolean: {str(e)}")
            return None

    def verify_insurance(self, insurance_qa: Dict[str, Any]) -> Dict[str, Any]:
        """Process insurance QA and extract structured information."""
        # Initialize verification result
        verify = {
            "eligibility": {
                "status": None,
                "effective_date": None,
                "plan_type": None,
                "group_number": None
            },
            "benefits": {
                "annual_maximum": None,
                "remaining_maximum": None,
                "deductible": None,
                "deductible_met": None,
                "benefit_period": None
            },
            "coverage": {
                "preventive": None,
                "basic": None,
                "major": None,
                "periodontics": None,
                "endodontics": None
            },
            "limitations": {
                "waiting_period": None,
                "frequency": None,
                "missing_tooth": None,
                "pre_authorization": None
            }
        }

        # Process each category and field
        for category, fields in insurance_qa.items():
            for field, qa in fields.items():
                if category in self.extract_helpers and field in self.extract_helpers[category]:
                    extractor = self.extract_helpers[category][field]
                    
                    if isinstance(qa, dict) and 'sample_answers' in qa and qa['sample_answers']:
                        idx = random.randint(0, 3)
                        sample_answer = qa['sample_answers'][idx]
                        verify[category][field] = extractor(sample_answer)

        return verify

# Usage example:
# Initialize the verifier
verifier = InsuranceVerifier(office_name='Raj dental clinic')

# Import your insurance_qa dictionary
from insurance_qa_sample import insurance_qa

# Process insurance verification
result = verifier.verify_insurance(insurance_qa)
print(result)

Error extracting boolean: 429 Resource has been exhausted (e.g. check quota).
{'eligibility': {'status': 'Active', 'effective_date': '03/15/2024', 'plan_type': 'DHMO', 'group_number': '345678'}, 'benefits': {'annual_maximum': 1000.0, 'remaining_maximum': 1200.0, 'deductible': 50.0, 'deductible_met': 50.0, 'benefit_period': 'Calendar Year'}, 'coverage': {'preventive': 100, 'basic': 75, 'major': 40, 'periodontics': 50, 'endodontics': 80}, 'limitations': {'waiting_period': None, 'frequency': 'Twice per year', 'missing_tooth': None, 'pre_authorization': True}}


In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from typing import Optional

class TestVerification:
    def __init__(self):
        # Initialize Gemini model
        load_dotenv()
        genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
        self.model = genai.GenerativeModel('gemini-1.5-flash-8b')
        self.chat = self.model.start_chat()

    def extract_boolean(self, question: str, response: str) -> Optional[bool]:
        """Extract yes/no answer from the response given a specific question."""
        print(f"\nDEBUG - Boolean Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")

        try:
            prompt = f"""
            Given this question and response, analyze the text to determine if it indicates a positive (yes) or negative (no) response.
            Consider common positive and negative indicators:

            Common positive indicators: yes, sure, affirmative, absolutely, of course, definitely, indeed, active, 
            eligible, covered, required, approved, confirmed, consent, positive, agree, proceed, what do you want, 
            what would you like, what would you like to do, what do you want to verify, what would you like to verify, 
            what can I help you with
            Common negative indicators: no, not, never, inactive, ineligible, not covered, not required, denied, rejected, 
            negative, disagree

            Note: Questions that ask for the next action or verification, such as 'Sure, what would you like to verify?' 
            or 'What do you want to do?' should be treated as positive (yes) responses.

            Question: "{question}"
            Response: "{response}"

            Return only one of the following: 'True' for positive, 'False' for negative, or 'None' if unclear. No other text.
            """

            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip().upper()  # Convert to uppercase for comparison

            print(f"Raw LLM response: '{llm_response.text}'")
            print(f"Cleaned result: '{result}'")

            # Check the result
            if result == 'TRUE':
                print("Returning True")
                return True
            elif result == 'FALSE':
                print("Returning False")
                return False
            else:
                print("Returning None")
                return None

        except Exception as e:
            print(f"Error in extract_boolean: {str(e)}")
            return None

def test_extract_boolean():
    # Create test instance
    verification = TestVerification()
    
    # Test cases
    test_cases = [
        ("Is the patient eligible?", "Yes, the patient is eligible", True),
        ("Would you mind verifying insurance coverage?", "Sure, what would you like to verify?", True),
        ("Is the patient eligible?", "No, the patient is not eligible", False),
        ("Would you mind verifying insurance coverage?", "No, I don't think that's necessary.", False),
        ("Is the coverage active?", "Coverage is active", True),
        ("Would you mind verifying insurance coverage?", "Of course, what would you like to verify?", True),
        ("Is the patient eligible?", "The patient is eligible.", True),
        ("Would you mind verifying insurance coverage?", "I'm not sure about that.", None),
        ("Would you mind verifying insurance coverage?", "Could you please repeat that?", None),
        ("Is the patient eligible?", "I'm not sure about that.", None)
    ]
    
    print("\nStarting boolean extraction tests...\n")
    
    for question, response, expected in test_cases:
        result = verification.extract_boolean(question, response)
        print(f"\nTest case: '{question}' with response: '{response}'")
        print(f"Expected: {expected}, Result: {result}")
        print("-" * 50)

if __name__ == "__main__":
    test_extract_boolean()



Starting boolean extraction tests...


DEBUG - Boolean Extraction:
Question: 'Is the patient eligible?'
Response: 'Yes, the patient is eligible'
Raw LLM response: 'True
'
Cleaned result: 'TRUE'
Returning True

Test case: 'Is the patient eligible?' with response: 'Yes, the patient is eligible'
Expected: True, Result: True
--------------------------------------------------

DEBUG - Boolean Extraction:
Question: 'Would you mind verifying insurance coverage?'
Response: 'Sure, what would you like to verify?'
Raw LLM response: 'True
'
Cleaned result: 'TRUE'
Returning True

Test case: 'Would you mind verifying insurance coverage?' with response: 'Sure, what would you like to verify?'
Expected: True, Result: True
--------------------------------------------------

DEBUG - Boolean Extraction:
Question: 'Is the patient eligible?'
Response: 'No, the patient is not eligible'
Raw LLM response: 'False
'
Cleaned result: 'FALSE'
Returning False

Test case: 'Is the patient eligible?' with response: 'N

# How the Dental AI Agent Code Pieces Come Together

This document explains how various code components work together in the dental AI agent to verify insurance information efficiently.

The main function coordinates all parts of the system to simulate a conversation with a patient for insurance verification. It uses natural language processing, speech recognition, and text-to-speech technologies.

## Main Components

1. **Libraries and Utilities:**
   - Libraries for environment management (`os`, `warnings`), data handling (`json`, `numpy`), and time management (`time`).
   - Utilities from custom modules to handle patient data, speech formatting, verification, and correction.

2. **Loading Models and Initializing Systems:**
   - **Speech Recognition:** Uses the `speech_recognition` library to capture and process spoken input.
   - **Whisper Model:** A pre-trained model used for transcribing speech to text.
   - **Text-to-Speech (TTS):** Converts text responses back into speech.
   - **Correction System:** Uses FAISS to enhance accent handling and verify spoken input.

3. **Initializing the AI Assistant:**
   - **Patient Information:** A simulated patient is generated for the demonstration.
   - **Insurance Verification:** The system initializes to handle verification tasks for the dental office.
   - **Conversation Flow Manager:** Manages the conversation flow, ensuring the dialogue stays on track and transitions smoothly between phases.

## Workflow

1. **Initial Greeting:**
   - The assistant starts with a professional greeting, introducing the dental office and asking for help with insurance verification.

2. **Main Conversation Loop:**
   - **Listening for Speech:** The system listens for spoken input using the microphone.
   - **Processing Speech:** Converts spoken input into text and checks if the text needs correction based on context.
   - **Handling Corrections:** If the input needs correction, the system enhances accent handling and may seek confirmation from the user.
   - **Responding:** The system processes the input and generates an appropriate response, which is then converted back to speech.

3. **Verification Completion:**
   - The system checks if all required verification information has been collected.
   - Displays and saves a summary of the verification results.

4. **Error Handling:**
   - The system handles common errors, such as misunderstood input, and requests the user to repeat or rephrase.

## Conclusion

The dental AI agent combines multiple technologies to create an efficient system for insurance verification. It leverages natural language processing to understand and respond to user input, ensuring accurate and professional interactions in a dental office setting.


In [7]:
# Part 1: Imports and Helper Functions
import os
import json
import re
import time
import warnings
import random
from datetime import datetime, timedelta

import nltk
import numpy as np
import speech_recognition as sr
import whisper
import torch
from TTS.api import TTS
import simpleaudio as sa
import pandas as pd
from dotenv import load_dotenv
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from scipy import signal
from typing import Optional

warnings.filterwarnings('ignore')

def format_date(date_str):
    """Format date for speech"""
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    return f"{date_obj.month}..{date_obj.day}..{date_obj.year}"

def fake_patient():
    """Generate fake patient data"""
    first_names = ['John', 'Maria', 'James', 'Sarah', 'Michael']
    last_names = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones']
    
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    
    return {
        'first_name': first_name,
        'last_name': last_name,
        'date_of_birth': format_date((datetime.now() - timedelta(days=random.randint(6570, 29200))).strftime('%Y-%m-%d')),
        'member_number': str(random.randint(100000000, 999999999)),
        'group_number': str(random.randint(1000, 9999)),
        'insurance_provider': random.choice(['Delta Dental', 'Cigna', 'MetLife', 'Aetna', 'Guardian'])
    }

def format_speech_output(text):
    """Format speech output"""
    text = text.replace(' ID ', ' I..D ').replace(' id ', ' I..D ')
    text = re.sub(r'(\d{1,2})/(\d{1,2})/(\d{4})', r'\1..\2..\3', text)
    
    # Format member ID
    text = re.sub(
        r'member I..D is (\d+)', 
        lambda m: f'member I..D is {" ".join(list(m.group(1)))}',
        text
    )
    
    # Format group number
    text = re.sub(
        r'group number is (\d+)',
        lambda m: f'group number is {" ".join(list(m.group(1)))}',
        text
    )
    
    # Format dollar amounts
    text = re.sub(
        r'\$(\d+(?:,\d{3})*(?:\.\d{2})?)',
        lambda m: f'$ {" ".join(list(m.group(1).replace(",", "")))}',
        text
    )
    
    # Format percentages
    text = re.sub(
        r'(\d+)%',
        lambda m: f'{" ".join(list(m.group(1)))} percent',
        text
    )
    
    return text

def get_verification_summary(verification):
    """Get verification status summary"""
    summary = {
        'collected': {},
        'missing': [],
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'status': 'incomplete'
    }
    
    for category in verification.verification_data:
        summary['collected'][category] = {}
        for field, value in verification.verification_data[category].items():
            if value is not None:
                summary['collected'][category][field] = value
            else:
                summary['missing'].append(f"{category}.{field}")
    
    if not summary['missing']:
        summary['status'] = 'complete'
    
    return summary

# Part 2: Speech Recognition and Handling Functions

def initialize_correction_system():
    """Initialize the speech correction system using FAISS and Instruct embeddings"""
    try:
        df = pd.read_csv("./correction_lookup.csv")
        embeddings = HuggingFaceInstructEmbeddings(
            model_name="hkunlp/instructor-base",
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True}
        )
        misheard_texts = df['misheard'].tolist()
        faiss_index = FAISS.from_texts(texts=misheard_texts, embedding=embeddings)
        print("Correction system initialized successfully")
        return faiss_index, df
    except Exception as e:
        print(f"Error initializing correction system: {str(e)}")
        return None, None

def find_similar_terms(query, faiss_index, df, current_context="patient information", threshold=0.75):
    """Search for similar terms using FAISS index with context"""
    if not query or not query.strip():
        return None, 0
        
    try:
        similar_docs_with_scores = faiss_index.similarity_search_with_score(query, k=1)
        if similar_docs_with_scores:
            doc, score = similar_docs_with_scores[0]
            confidence = 1 / (1 + score)
            
            # Get row with correction and context
            matched_row = df[df['misheard'] == doc.page_content].iloc[0]
            
            print(f"\nDEBUG - FAISS Search:")
            print(f"Query: '{query}'")
            print(f"Matched: '{doc.page_content}'")
            print(f"Correction: '{matched_row['correction']}'")
            print(f"Context: '{matched_row['context']}'")
            print(f"Current context: '{current_context}'")
            print(f"Confidence: {confidence}")
            
            # Only return if context matches
            if matched_row['context'].lower() == current_context.lower():
                return {'original': query, 'correction': matched_row['correction']}, confidence
            else:
                print("Skipping correction - context mismatch")
                return None, 0
                
    except Exception as e:
        print(f"Error in similarity search: {str(e)}")
    return None, 0

def get_llm_correction(text: str, verification) -> dict:
    """Use LLM to suggest correction when lookup.csv doesn't have a match"""
    try:
        prompt = f"""In an insurance verification context, suggest the most likely correction for this input.
        
        Common insurance verification phrases:
        - Questions about patient info: "What is the patient's name?", "What is the date of birth?"
        - Insurance queries: "What is the member ID?", "What's the eligibility status?"
        - Coverage questions: "What's the annual maximum?", "What's the deductible?"
        
        Input: "{text}"
        
        Return JSON format:
        {{"suggestion": "corrected phrase",
          "confidence": "high/medium/low"}}
        
        Return only valid JSON, no other text."""
        
        response = verification.chat.send_message({"text": prompt})
        result = json.loads(response.text.strip())
        
        return result
            
    except Exception as e:
        print(f"Error in LLM correction: {str(e)}")
        return None

# def validate_input_context(text: str, verification, faiss_index=None, correction_df=None) -> dict:
#     """Validate if input needs correction in insurance verification context"""
#     try:
#         print(f"\nDEBUG - Validation:")
#         print(f"Input text: '{text}'")
        
#         # First check if the input makes sense in insurance verification context
#         prompt = f"""Determine if this input makes sense in an insurance verification conversation context which might not be
#         semantically correct always. The input should be processed as-is if:
#         1. It's almost grammatically correct question or statement with some missing words in between
#         2. It's a common response in conversation (yes, sure, of course, etc.)
#         3. It contains insurance-related terms used correctly
#         4. It's a reasonable response to questions about verification
#         5. It's a complete or partial sentence that conveys meaning
#         6. It contains any insurance-related information
#         7. It's a common conversational response
#         8. Even if informal, the meaning is clear
#         9. If its question about patient info, it has to semantically make sense so use faiss correction
        
#         Only needs correction if:
#         1. Contains misheard/mispronounced words (like "data birth" instead of "date of birth")
#         2. Uses incorrect terms that change meaning (like "legible" instead of "eligible")
#         3. Has significant grammar errors that make it unclear
        
#         Examples that make sense (no correction needed):
#         - Questions: "What would you like to verify?", "What do you need?"
#         - Responses: "Of course", "Yes, I can help", "The patient is eligible with active status"
#         - Statements: "I need to check the system", "Let me verify that for you"
#         - Help phrases: "How may help you", "How can I assist", "What would you like to verify"
        
#         Return ONLY 'True' if input makes sense, 'False' if unclear/needs correction.
#         No other text.
        
#         Text: "{text}" """
        
#         response = verification.chat.send_message({"text": prompt})
#         makes_sense = response.text.strip().lower() == 'true'
#         print(f"Makes semantic sense: {makes_sense}")
        
#         if makes_sense:
#             return {"needs_correction": False, "reason": "Valid in verification context"}

#         # Only try FAISS and further correction if input doesn't make sense
#         if faiss_index is not None and correction_df is not None:
#             correction, confidence = find_similar_terms(text, faiss_index, correction_df)
#             print(f"FAISS confidence: {confidence}")
#             if correction and confidence >= 0.8:  # Only use high confidence corrections
#                 return {
#                     "needs_correction": True,
#                     "reason": f"FAISS correction (confidence: {confidence:.2f}): {correction['correction']}"
#                 }

#         # Default to not correcting if unsure
#         return {"needs_correction": False, "reason": "Proceeding without correction"}

#     except Exception as e:
#         print(f"Error in validation: {str(e)}")
#         return {"needs_correction": False, "reason": f"Error in validation: {str(e)}"}

def validate_input_context(text: str, verification, faiss_index=None, correction_df=None) -> dict:
    try:
        # Get current state
        verification_started = getattr(verification, 'verification_started', False)
        verification_asked = hasattr(verify_patient, 'verification_asked')
        current_context = "insurance verification" if verification_started else "patient information"
        
        print(f"\nDEBUG - Validation:")
        print(f"Input text: '{text}'")
        print(f"Verification started: {verification_started}")
        print(f"Current context: {current_context}")

        # Insurance verification phase
        if verification_started:
            current_category = getattr(verification.conversation_manager, 'current_category', None)
            if current_category:
                current_fields = verification.conversation_manager.insurance_qa[current_category].keys()
                current_field = None
                
                for field in current_fields:
                    if verification.verification_data[current_category][field] is None:
                        current_field = field
                        break

                if current_field:
                    # Get current question
                    current_question = verification.conversation_manager.insurance_qa[current_category][current_field]['question']
                    print(f"Current field: {current_field}")
                    print(f"Current question: {current_question}")

                    # First try to extract information
                    info_prompt = f"""Does this response contain information relevant to: {current_question}
                    Look for:
                    1. Dollar amounts for payment questions
                    2. Percentages for coverage questions
                    3. Dates for effective date/period questions
                    4. Status terms (active, eligible) for eligibility
                    5. Any numbers or terms that answer the question
                    6. Even in incomplete sentences, is the required info present?
                    
                    Return ONLY 'True' if extractable info exists, 'False' if not.
                    Text: "{text}" """
                    
                    response = verification.chat.send_message({"text": info_prompt})
                    has_info = response.text.strip().lower() == 'true'
                    print(f"Contains relevant info for {current_field}: {has_info}")
                    
                    if has_info:
                        return {"needs_correction": False, "reason": "Contains relevant information"}

                    # If no info found, try FAISS correction
                    if faiss_index is not None and correction_df is not None:
                        correction, confidence = find_similar_terms(text, faiss_index, correction_df, current_context)
                        if correction and confidence >= 0.8:
                            return {
                                "needs_correction": True,
                                "reason": f"Insurance verification correction found: {correction['correction']}"
                            }
                    return {"needs_correction": False, "reason": "Ask to rephrase"}

            return {"needs_correction": False, "reason": "Process as conversation response"}

        # Patient information phase
        else:
            if faiss_index is not None and correction_df is not None:
                correction, confidence = find_similar_terms(text, faiss_index, correction_df, current_context)
                if correction and confidence >= 0.8:
                    if verification_asked:
                        return {
                            "needs_correction": True,
                            "reason": f"Consent response correction found: {correction['correction']}"
                        }
                    else:
                        return {
                            "needs_correction": True,
                            "reason": f"Patient information correction found: {correction['correction']}"
                        }
            return {"needs_correction": False, "reason": "No correction needed"}

    except Exception as e:
        print(f"Error in validation: {str(e)}")
        return {"needs_correction": False, "reason": f"Error in validation: {str(e)}"}
    
# def enhance_accent_handling(text, faiss_index, correction_df, verification):
#     """Enhanced accent handling using FAISS for corrections with LLM fallback"""
#     if not text or not text.strip():
#         return {'original': text, 'corrected': text, 'needs_confirmation': False}
        
#     cleaned_text = text.lower().replace('?', ',').replace(' and ', ', ').replace('please', '')
#     requests = [req.strip() for req in cleaned_text.split(',') if req.strip()]
    
#     corrections_made = []
#     needs_confirmation = False
#     corrected_requests = []
    
#     for request in requests:
#         # First try FAISS lookup
#         correction, confidence = find_similar_terms(request, faiss_index, correction_df)
        
#         if correction:
#             corrected_requests.append(correction['correction'])
#             corrections_made.append((request, correction['correction'], confidence))
#             needs_confirmation = needs_confirmation or confidence < 0.75
#         else:
#             # If no FAISS match, try LLM suggestion
#             correction = get_llm_correction(request, verification)
#             if correction:
#                 corrected_requests.append(correction['suggestion'])
#                 # Map confidence levels to numerical values
#                 confidence_map = {'high': 0.8, 'medium': 0.5, 'low': 0.3}
#                 confidence = confidence_map.get(correction['confidence'], 0.5)
#                 corrections_made.append((request, correction['suggestion'], confidence))
#                 needs_confirmation = True  # Always confirm LLM suggestions
#             else:
#                 corrected_requests.append(request)
                
#     seen = set()
#     corrected_requests = [x for x in corrected_requests if not (x in seen or seen.add(x))]
#     corrected_text = ', '.join(corrected_requests)
    
#     confirmation_msg = None
#     if needs_confirmation:
#         confirmation_msg = "I heard: " + ", ".join(
#             f"'{orig}' (did you mean '{corr}'?)" 
#             for orig, corr, conf in corrections_made
#         )
        
#     return {
#         'original': text,
#         'corrected': corrected_text,
#         'needs_confirmation': needs_confirmation,
#         'confirmation_msg': confirmation_msg,
#         'corrections': corrections_made
#     }

def enhance_accent_handling(text, faiss_index, correction_df, verification):
    """Enhanced accent handling using FAISS corrections"""
    if not text or not text.strip():
        return {'original': text, 'corrected': text, 'needs_confirmation': False}
    
    # Determine current context based on verification state
    verification_started = getattr(verification, 'verification_started', False)
    has_current_category = hasattr(verification, 'current_category')
    in_verification = verification_started and has_current_category
    current_context = "insurance verification" if in_verification else "patient information"
    
    print(f"\nDEBUG - Accent Handling:")
    print(f"Original text: '{text}'")
    print(f"Current context: {current_context}")
        
    cleaned_text = text.lower().replace('?', ',').replace(' and ', ', ').replace('please', '')
    requests = [req.strip() for req in cleaned_text.split(',') if req.strip()]
    
    corrections_made = []
    needs_confirmation = False
    corrected_requests = []
    
    for request in requests:
        # Try FAISS lookup with context
        correction, confidence = find_similar_terms(request, faiss_index, correction_df, current_context)
        print(f"Processing request: '{request}' in context: {current_context}")
        
        if correction:
            corrected_requests.append(correction['correction'])
            corrections_made.append((request, correction['correction'], confidence))
            needs_confirmation = needs_confirmation or confidence < 0.75
            print(f"FAISS correction found: '{correction['correction']}' with confidence: {confidence}")
        else:
            # If no FAISS match, keep original
            corrected_requests.append(request)
            print(f"No correction found for: '{request}'")
                
    seen = set()
    corrected_requests = [x for x in corrected_requests if not (x in seen or seen.add(x))]
    corrected_text = ', '.join(corrected_requests)
    
    confirmation_msg = None
    if needs_confirmation:
        confirmation_msg = "I heard: " + ", ".join(
            f"'{orig}' (did you mean '{corr}'?)" 
            for orig, corr, conf in corrections_made
        )
    
    print(f"Final corrected text: '{corrected_text}'")
    return {
        'original': text,
        'corrected': corrected_text,
        'needs_confirmation': needs_confirmation,
        'confirmation_msg': confirmation_msg,
        'corrections': corrections_made
    }

def initialize_enhanced_recognition():
    """Initialize speech recognition with enhanced settings"""
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 3000
    recognizer.dynamic_energy_threshold = True
    recognizer.dynamic_energy_adjustment_damping = 0.15
    recognizer.dynamic_energy_ratio = 1.5
    recognizer.pause_threshold = 1.0
    
    def remove_noise(audio_data):
        audio_array = np.frombuffer(audio_data.frame_data, dtype=np.int16)
        nyquist = 22050 / 2
        low = 300 / nyquist
        high = 3000 / nyquist
        b, a = signal.butter(4, [low, high], btype='band')
        filtered_audio = signal.filtfilt(b, a, audio_array)
        return filtered_audio
    
    recognizer.remove_noise = remove_noise
    return recognizer

def listen_for_speech(recognizer, source, play_obj=None):
    """Listen for speech with interruption handling"""
    print("\nListening...")
    if play_obj and play_obj.is_playing():
        play_obj.stop()
        
    try:
        full_audio_data = b''
        is_speaking = False
        
        while True:
            try:
                audio = recognizer.listen(source, timeout=0.5)
                if len(audio.get_raw_data()) > 0:
                    is_speaking = True
                    full_audio_data += audio.get_raw_data()
                
                if is_speaking:
                    try:
                        recognizer.listen(source, timeout=1.0)
                        break
                    except sr.WaitTimeoutError:
                        continue
            except sr.WaitTimeoutError:
                if full_audio_data:
                    break
                continue
                
        if not full_audio_data:
            return None
            
        return sr.AudioData(full_audio_data, audio.sample_rate, audio.sample_width)
        
    except Exception as e:
        print(f"Error in listening: {str(e)}")
        return None

def handle_speech_output(queue, play_obj, wav_data, recognizer, source):
    """Handle speech output with interruption detection"""
    try:
        wav_norm = wav_data * (32767 / max(0.01, np.max(np.abs(wav_data))))
        audio_obj = sa.WaveObject(wav_norm.astype(np.int16), 1, 2, 22050)
        
        if not play_obj or not play_obj.is_playing():
            play_obj = audio_obj.play()
            
            while play_obj.is_playing():
                try:
                    if recognizer.get_energy() > 3000:
                        play_obj.stop()
                        queue.clear()
                        return None
                    time.sleep(0.1)
                except:
                    continue
                    
            return play_obj
        else:
            queue.append(audio_obj)
            return play_obj
            
    except Exception as e:
        print(f"Error in speech output: {str(e)}")
        return play_obj

def handle_confirmation(text):
    """Handle confirmation responses"""
    text_lower = text.lower()
    return (
        True if any(x in text_lower for x in ['yes', 'correct', 'right', 'yeah', 'yep']) 
        else False if any(x in text_lower for x in ['no', 'incorrect', 'wrong', 'nope']) 
        else None
    )

# Part 3: Insurance Verification Class

class InsuranceVerification:
    def __init__(self, office_name, patient_data):
        self.office_name = office_name
        self.patient_data = patient_data
        self.verification_data = {
            'eligibility': {
                'status': None,
                'effective_date': None,
                'plan_type': None
            },
            'benefits': {
                'annual_maximum': None,
                'remaining_maximum': None,
                'deductible': None,
                'deductible_met': None,
                'benefit_period': None
            },
            'coverage': {
                'preventive': None,
                'basic': None,
                'major': None,
                'periodontics': None,
                'endodontics': None
            },
            'limitations': {
                'waiting_period': None,
                'frequency': None,
                'missing_tooth': None,
                'pre_authorization': None
            }
        }
        
        # Initialize extraction functions for each field
        self.extraction_functions = {
            'eligibility': {
                'status': self.extract_status,
                'effective_date': self.extract_date,
                'plan_type': self.extract_plan_type
            },
            'benefits': {
                'annual_maximum': self.extract_amount,
                'remaining_maximum': self.extract_amount,
                'deductible': self.extract_amount,
                'deductible_met': self.extract_amount,
                'benefit_period': self.extract_period
            },
            'coverage': {
                'preventive': self.extract_percentage,
                'basic': self.extract_percentage,
                'major': self.extract_percentage,
                'periodontics': self.extract_percentage,
                'endodontics': self.extract_percentage
            },
            'limitations': {
                'waiting_period': self.extract_period,
                'frequency': self.extract_frequency,
                'missing_tooth': self.extract_boolean,
                'pre_authorization': self.extract_boolean
            }
        }
        
        # Initialize Gemini model
        load_dotenv()
        genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
        self.model = genai.GenerativeModel('gemini-1.5-flash-8b')
        self.chat = self.model.start_chat()

    def extract_status(self, response: str, question: str) -> Optional[str]:
        """Extract insurance status from response"""
        print(f"\nDEBUG - Status Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")
        
        try:
            prompt = f"""Given this question and response, determine the insurance status.
            Question: "{question}"
            Response: "{response}"
            
            Return only 'Active' if patient is eligible/active/covered,
            'Inactive' if not eligible/inactive/not covered,
            or 'None' if unclear.

            Examples:
            Q. "What is the patient's current eligibility status?" A: "The patient is currently active and eligible for benefits." -> "Active"
            Q. "What is the patient's current eligibility status?" A: "The patient's coverage is active and verified." -> "Active"
            Q. "What is the patient's current eligibility status?" A: "Yes, the patient is eligible and the coverage is in force." -> "Active"
            Q. "What is the patient's current eligibility status?" A: "The patient is eligible with active coverage status." -> "Inactive"
            """
            
            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip()
            print(f"LLM status result: '{result}'")
            
            if result in ['Active', 'Inactive']:
                return result
            return None
                
        except Exception as e:
            print(f"Error extracting status: {str(e)}")
            return None


    def extract_date(self, response: str, question: str) -> Optional[str]:
        """Extract date in MM/DD/YYYY format from the response given a specific question."""
        print(f"\nDEBUG - Date Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")

        current_year = datetime.now().year

        try:
            prompt = f"""
            Given this question and response, extract the date and convert it to MM/DD/YYYY format.
            If the response refers to 'calendar year' or 'year end', use the current year ({current_year}) to determine the date.

            Question: "{question}"
            Response: "{response}"
            
            Examples:
            Q: "What is their effective date of coverage?" A: "The coverage effective date is January 1st, {current_year}." -> "01/01/{current_year}"
            Q: "What is their effective date of coverage?" A: "Their coverage began on March 15th, {current_year}." -> "03/15/{current_year}"
            Q: "What is their effective date of coverage?" A: "The effective date shows as September 1st, {current_year - 1}." -> "09/01/{current_year - 1}"
            Q: "What is their effective date of coverage?" A: "Coverage has been effective since July 1st, {current_year}." -> "07/01/{current_year}"
            Q: "What is their effective date of coverage?" A: "Coverage has been effective since the start of the year." -> "01/01/{current_year}"
            Q: "What is their benefit period?" A: "The benefit period is calendar year, January through December." -> "12/31/{current_year}"
            Q: "What is their benefit period?" A: "Benefits run on a fiscal year, July through June." -> "06/30/{current_year + 1}"
            Q: "When did the patient's treatment start?" A: "The treatment started on May 5th, {current_year}." -> "05/05/{current_year}"
            Q: "What was the date of service?" A: "The date of service was August 20th, {current_year}." -> "08/20/{current_year}"
            """

            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip()
            print(f"LLM date result: '{result}'")

            if result == 'None':
                return None

            # Validate the response looks like a date
            parts = result.split('/')
            if len(parts) == 3 and all(p.isdigit() for p in parts):
                return result

            return None

        except Exception as e:
            print(f"Error extracting date: {str(e)}")
            return None


    def extract_amount(self, response: str, question: str) -> Optional[float]:
        """Extract monetary amount from response"""
        print(f"\nDEBUG - Amount Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")
        
        try:
            prompt = f"""Given this question and response, extract the dollar amount.
            Question: "{question}"
            Response: "{response}"
            
            Return only the number (no $ or commas) or 'None' if no amount found.
            
            Examples:
            Q: "What is the deductible amount?" A: "fifty dollars" -> "50"
            Q: "What is their annual maximum benefit?" A: "1k" -> "1000"
            Q: "How much deductible has been met?" A: "They haven't met any of the deductible yet." -> "0"
            """
            
            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip()
            print(f"LLM amount result: '{result}'")
            
            if result == 'None':
                return None
                
            try:
                return float(result)
            except ValueError:
                return None
                
        except Exception as e:
            print(f"Error extracting amount: {str(e)}")
            return None

    def extract_percentage(self, response: str, question: str) -> Optional[int]:
        """Extract percentage from the response given a specific question."""
        print(f"\nDEBUG - Percentage Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")

        try:
            prompt = f"""
            Given this question and response, extract the percentage and return only the number (no % symbol) or 'None' if no percentage is found.
            Interpret phrases like 'half coverage' as 50% and 'full coverage' as 100%.
            
            Question: "{question}"
            Response: "{response}"
            
            Examples:
            Q: "What is the coverage percentage for preventive services?" A: "Preventive services are covered at 100%." -> "100"
            Q: "What about basic services?" A: "Basic services are covered at 80%." -> "80"
            Q: "What is the coverage for major services?" A: "Major services are covered at 50%." -> "50"
            Q: "What's the coverage for periodontal services?" A: "Periodontal services are covered at 80%." -> "80"
            Q: "What is the coverage percentage for preventive services?" A: "Preventive services have full coverage." -> "100"
            Q: "What about basic services?" A: "Basic services have half coverage." -> "50"
            """

            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip()
            print(f"LLM percentage result: '{result}'")

            if result == 'None':
                return None

            try:
                percentage = int(result)
                if 0 <= percentage <= 100:
                    return percentage
                return None
            except ValueError:
                return None

        except Exception as e:
            print(f"Error extracting percentage: {str(e)}")
            return None


    def extract_plan_type(self, response: str, question: str) -> Optional[str]:
        """Extract insurance plan type from the response given a specific question."""
        print(f"\nDEBUG - Plan Type Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")

        try:
            prompt = f"""
            Given this question and response, extract the insurance plan type.
            Common types include: PPO, HMO, DHMO, Indemnity, etc.
            Return only the plan type or 'None' if unclear. No other text.

            Question: "{question}"
            Response: "{response}"
            
            Examples:
            Q: "What type of plan do they have?" A: "This is a PPO plan." -> "PPO"
            Q: "What type of plan do they have?" A: "They have a DHMO plan." -> "DHMO"
            Q: "What type of plan do they have?" A: "It's an indemnity plan." -> "Indemnity"
            Q: "What type of plan do they have?" A: "The patient has a PPO Plus plan." -> "PPO Plus"
            """

            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip()
            print(f"LLM plan type result: '{result}'")

            if result == 'None':
                return None
            return result

        except Exception as e:
            print(f"Error extracting plan type: {str(e)}")
            return None


    def extract_group_number(self, response: str, question: str) -> Optional[str]:
        """Extract group number from the response given a specific question."""
        print(f"\nDEBUG - Group Number Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")

        try:
            prompt = f"""
            Given this question and response, extract the insurance group number.
            The group number might be labeled as 'Group #', 'Group Number', or similar.
            Return only the group number or 'None' if not found. No other text.

            Question: "{question}"
            Response: "{response}"
            
            Examples:
            Q: "Could you verify their group number?" A: "The group number is 123456." -> "123456"
            Q: "Could you verify their group number?" A: "I'm showing group number 789012." -> "789012"
            Q: "Could you verify their group number?" A: "Yes, the group number is 345678." -> "345678"
            Q: "Could you verify their group number?" A: "The verified group number is 901234." -> "901234"
            """

            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip()
            print(f"LLM group number result: '{result}'")

            if result == 'None':
                return None
            return result

        except Exception as e:
            print(f"Error extracting group number: {str(e)}")
            return None


    def extract_period(self, response: str, question: str) -> Optional[str]:
        """Extract benefit or waiting period from the response given a specific question."""
        print(f"\nDEBUG - Period Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")

        try:
            prompt = f"""
            Given this question and response, extract the time period.
            Common formats include: 'Calendar Year', 'Contract Year', '6 months', '12 months', etc.
            Return only the period or 'None' if not found. No other text.

            Question: "{question}"
            Response: "{response}"
            
            Examples:
            Q: "What is their benefit period?" A: "The benefit period is calendar year, January through December." -> "Calendar Year"
            Q: "What is their benefit period?" A: "Benefits run on a fiscal year, July through June." -> "Fiscal Year"
            Q: "What is their benefit period?" A: "It's a calendar year benefit period." -> "Calendar Year"
            Q: "What is their benefit period?" A: "The benefit period follows the calendar year." -> "Calendar Year"
            """

            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip()
            print(f"LLM period result: '{result}'")

            if result == 'None':
                return None
            return result

        except Exception as e:
            print(f"Error extracting period: {str(e)}")
            return None


    def extract_frequency(self, response: str, question: str) -> Optional[dict]:
        """Extract frequency limitations from the response given a specific question."""
        print(f"\nDEBUG - Frequency Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")

        try:
            prompt = f"""
            Given this question and response, extract the frequency limitation for each type of service mentioned.
            Common formats include: 'Once every 6 months', '2 per year', 'Annual', etc.
            Return a dictionary with the type of service as the key and the frequency as the value. If unable to extract, return the original response as the value.

            Question: "{question}"
            Response: "{response}"
            
            Examples:
            Q: "What are the frequency limitations?" A: "Cleanings are covered twice per calendar year." -> {{"Cleanings": "twice per calendar year"}}
            Q: "What are the frequency limitations?" A: "Exams and cleanings twice per year, x-rays once every 3 years." -> {{"Exams": "twice per year", "Cleanings": "twice per year", "X-rays": "once every 3 years"}}
            Q: "What are the frequency limitations?" A: "Two cleanings per year with 6 months separation required." -> {{"Cleanings": "twice per year, 6 months separation"}}
            Q: "What are the frequency limitations?" A: "Comprehensive exams once every 3 years, routine exams twice per year." -> {{"Comprehensive exams": "once every 3 years", "Routine exams": "twice per year"}}
            """

            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip()
            print(f"LLM frequency result: '{result}'")

            if result == 'None':
                return {"Original Response": response}
            
            try:
                # Convert the string representation of a dictionary to an actual dictionary
                frequency_dict = eval(result)
                return frequency_dict
            except (SyntaxError, ValueError):
                return {"Original Response": response}

        except Exception as e:
            print(f"Error extracting frequency: {str(e)}")
            return {"Original Response": response}

    def extract_boolean(self, question: str, response: str) -> Optional[bool]:
        """Extract yes/no answer from the response given a specific question."""
        print(f"\nDEBUG - Boolean Extraction:")
        print(f"Question: '{question}'")
        print(f"Response: '{response}'")

        try:
            prompt = f"""
            Given this question and response, analyze the text to determine if it indicates a positive (yes) or negative (no) response.
            Consider common positive and negative indicators:

            Common positive indicators: yes, sure, affirmative, absolutely, of course, definitely, indeed, active, 
            eligible, covered, required, approved, confirmed, consent, positive, agree, proceed, what do you want, 
            what would you like, what would you like to do, what do you want to verify, what would you like to verify, 
            what can I help you with
            Common negative indicators: no, not, never, inactive, ineligible, not covered, not required, denied, rejected, 
            negative, disagree

            Note: Questions that ask for the next action or verification, such as 'Sure, what would you like to verify?' 
            or 'What do you want to do?' should be treated as positive (yes) responses.

            Question: "{question}"
            Response: "{response}"

            Return only one of the following: 'True' for positive, 'False' for negative, or 'None' if unclear. No other text.
            """

            llm_response = self.chat.send_message({"text": prompt})
            result = llm_response.text.strip().upper()  # Convert to uppercase for comparison

            print(f"Raw LLM response: '{llm_response.text}'")
            print(f"Cleaned result: '{result}'")

            # Check the result
            if result == 'TRUE':
                print("Returning True")
                return True
            elif result == 'FALSE':
                print("Returning False")
                return False
            else:
                print("Returning None")
                return None

        except Exception as e:
            print(f"Error in extract_boolean: {str(e)}")
            return None


def verify_patient(patient_data, query):
    if not hasattr(verify_patient, 'asked_fields'):
        verify_patient.asked_fields = set()
    if not hasattr(verify_patient, 'provided_fields'):
        verify_patient.provided_fields = set()

    query = query.lower().replace('?', ',').replace(' and ', ', ').replace('please', '').strip()
    requests = [req.strip() for req in query.split(',') if req.strip()]
    
    formatted_responses = {
        'name': f"{patient_data['first_name']} {patient_data['last_name']}",
        'date_of_birth': patient_data['date_of_birth'],
        'member_id': patient_data['member_number']
    }

    # First check if it's a name request
    if any(word in query for word in ['name', 'who is', 'patient name']):
        return f"The patient's name is {formatted_responses['name']}"
        
    response_parts = []
    # Process only the first relevant request
    for request in requests:
        if not response_parts:  # Only process if we haven't added a response yet
            if ('date' in request or 'birth' in request or 'dob' in request):
                verify_patient.asked_fields.add('dob')
                verify_patient.provided_fields.add('dob')
                response_parts.append(f"The date of birth is {formatted_responses['date_of_birth']}")
            elif ('member' in request or 'id' in request):
                verify_patient.asked_fields.add('member_id')
                verify_patient.provided_fields.add('member_id')
                response_parts.append(f"The member I..D is {formatted_responses['member_id']}")

    print(f"Current asked fields: {verify_patient.asked_fields}")
    print(f"Current provided fields: {verify_patient.provided_fields}")
    
    # Just return the response without asking about verification
    if len(response_parts) > 0:
        return " ".join(response_parts)
    return "What information would you like about the patient?"

class ConversationFlowManager:
    def __init__(self, verification, patient):
        self.verification = verification
        self.patient = patient
        self.is_patient_info_phase = True     # Start in patient info phase
        self.verification_started = False      # Insurance verification not started yet
        self.current_category = 'eligibility'  # Start with eligibility category
        
        # Define questions for each field
        self.insurance_qa = {
            'eligibility': {
                'status': {
                    'question': "What is the patient's current eligibility status?"
                },
                'effective_date': {
                    'question': "What is the effective date of coverage?"
                },
                'plan_type': {
                    'question': "What type of plan does the patient have?"
                }
            },
            'benefits': {
                'annual_maximum': {
                    'question': "What is the annual maximum benefit?"
                },
                'remaining_maximum': {
                    'question': "What is the remaining benefit amount?"
                },
                'deductible': {
                    'question': "What is the deductible amount?"
                },
                'deductible_met': {
                    'question': "How much of the deductible has been met?"
                },
                'benefit_period': {
                    'question': "What is the benefit period?"
                }
            },
            'coverage': {
                'preventive': {
                    'question': "What is the coverage percentage for preventive services?"
                },
                'basic': {
                    'question': "What is the coverage percentage for basic services?"
                },
                'major': {
                    'question': "What is the coverage percentage for major services?"
                },
                'periodontics': {
                    'question': "What is the coverage percentage for periodontal services?"
                },
                'endodontics': {
                    'question': "What is the coverage percentage for endodontic services?"
                }
            },
            'limitations': {
                'waiting_period': {
                    'question': "Are there any waiting periods?"
                },
                'frequency': {
                    'question': "What are the frequency limitations?"
                },
                'missing_tooth': {
                    'question': "Is there a missing tooth clause?"
                },
                'pre_authorization': {
                    'question': "Are there any pre-authorization requirements?"
                }
            }
        }
        
        # Order of categories to verify
        self.categories_order = ['eligibility', 'benefits', 'coverage', 'limitations']
    
    def check_transition_state(self, text: str) -> bool:
        """Use LLM to determine conversation state and readiness to transition"""
        try:
            # First check if we have all required fields
            required_fields = {'dob', 'member_id'}
            fields_collected = (hasattr(verify_patient, 'provided_fields') and 
                            verify_patient.provided_fields >= required_fields)
            
            if not fields_collected:
                print("Cannot transition: Required fields not collected")
                return False
                
            # Check for transition phrases directly first
            transition_phrases = [
                "how may i help",
                "how can help",
                "what can i help",
                "how can i assist",
                "what else",
                "what's next",
                "ready",
                "let's proceed",
                "go ahead"
            ]
            text_lower = text.lower()
            
            # If it's a direct match with transition phrases
            if any(phrase in text_lower for phrase in transition_phrases):
                print("Transition phrase detected, proceeding to verification")
                return True
                
            # If no direct match, use LLM as backup
            prompt = f"""You must respond with ONLY 'transition' or 'continue'.
            
            Determine if this response indicates readiness to proceed with insurance verification.
            The person has already provided date of birth and member ID. Do not worry about semantic correctness
            as long as the response makes sense.            
            Common transition phrases include:
            - Offering help (e.g., "how may I help")
            - Asking what's next
            - Indicating readiness to proceed
            - Expressing availability to assist
            
            Input: {text}
            
            Respond ONLY with:
            'transition' - if they're ready to proceed
            'continue' - if they're not indicating readiness"""
            
            response = self.verification.chat.send_message({"text": prompt})
            result = response.text.strip().lower()
            result = result.replace('```', '').strip()
            
            print(f"Required fields collected: {verify_patient.provided_fields}")
            print(f"Transition check result: {result}")
            
            should_transition = result == 'transition'
            if should_transition:
                print("LLM detected transition readiness")
            
            return should_transition
                
        except Exception as e:
            print(f"Error in transition check: {str(e)}")
            return False
            
    def process_response(self, text, verification):
        print(f"\n=== DEBUG - Process Response Start ===")
        print(f"Input text: '{text}'")
        print(f"Current phase: {'Patient Info' if self.is_patient_info_phase else 'Insurance Verification'}")

        if self.is_patient_info_phase:
            if hasattr(verify_patient, 'verification_asked'):
                print("Checking verification consent")
                question = "Would you mind verifying patient insurance coverage?"
                consent = verification.extract_boolean(question, text)
                print(f"Consent: {consent}")
                
                if consent:
                    print("Consent received - starting verification")
                    delattr(verify_patient, 'verification_asked')
                    self.is_patient_info_phase = False
                    verification.verification_started = True
                    verification.conversation_manager = self  # Pass self as conversation manager
                    next_question = self.get_next_question()
                    return next_question, True
                elif consent is False:
                    verification.verification_started = False
                    return "I understand. Please let me know when you're ready to proceed.", False
            
            required_fields = {'dob', 'member_id'}
            if (hasattr(verify_patient, 'provided_fields') and 
                verify_patient.provided_fields >= required_fields):

                help_prompt = f"""Is this a phrase offering help or asking how to assist? They usually ask these after
                providing patient name, dob and member id. The sentence does not have to be semantically correct as long
                as we can interprete it as an offering of help.
                Examples:
                - "How may I help you?"
                - "What can help you with?"
                - "How can I assist?"
                
                Return only 'True' or 'False'.
                Text: {text}"""
                
                is_help_phrase = verification.chat.send_message({"text": help_prompt}).text.strip().lower() == 'true'
                print(f"Is help phrase? {is_help_phrase}")

                if is_help_phrase:
                    print("Help phrase detected - asking for verification")
                    setattr(verify_patient, 'verification_asked', True)
                    return "Would you mind verifying patient insurance coverage?", False

            patient_info_response = verify_patient(verification.patient_data, text)
            return patient_info_response, False

        # Rest of insurance verification phase stays the same
        print("\n=== DEBUG - Insurance Verification Phase ===")
        print(f"Current category: {self.current_category}")
        current_fields = self.insurance_qa[self.current_category].keys()
        
        for field in current_fields:
            if verification.verification_data[self.current_category][field] is None:
                question = self.insurance_qa[self.current_category][field]['question']
                extract_func = verification.extraction_functions[self.current_category][field]
                
                value = extract_func(text, question)
                
                if value is not None:
                    verification.verification_data[self.current_category][field] = value
                    next_question = self.get_next_question()
                    print(f"Extracted {field}: {value}")
                    if next_question:
                        return next_question, False
                    else:
                        return "Verification complete!", False

        print("=== DEBUG - Process Response End ===")
        return "I didn't quite get that. Could you rephrase?", False

    def get_next_question(self):
        """Get next verification question based on current state"""
        if not self.verification_started:
            self.verification_started = True
            # First question of verification
            return self.insurance_qa['eligibility']['status']['question']

        current_fields = self.insurance_qa[self.current_category].keys()
        
        # Look for next empty field in current category
        for field in current_fields:
            if self.verification.verification_data[self.current_category][field] is None:
                return self.insurance_qa[self.current_category][field]['question']

        # All fields in current category complete, move to next category
        current_index = self.categories_order.index(self.current_category)
        if current_index < len(self.categories_order) - 1:
            next_category = self.categories_order[current_index + 1]
            self.current_category = next_category
            return (
                self._get_category_intro() +
                self.insurance_qa[next_category][list(self.insurance_qa[next_category].keys())[0]]['question']
            )

        return None  # All categories complete

    def _get_category_intro(self):
        """Get introduction message for new category"""
        intros = {
            'eligibility': "Let's verify eligibility. ",
            'benefits': "Now for benefits. ",
            'coverage': "Let's check coverage percentages. ",
            'limitations': "Finally, about limitations. "
        }
        return intros.get(self.current_category, "")     



def main():
    """Main function with improved conversation flow and speech formatting"""
    try:
        # Initialize basic components
        nltk.download('punkt_tab', quiet=True)
        office_name = "Everest Dental Clinic"

        # Create instances
        patient = fake_patient()
        verification = InsuranceVerification(office_name, patient)
        flow_manager = ConversationFlowManager(verification, patient)
        faiss_index, correction_df = initialize_correction_system()
        
        if faiss_index is None or correction_df is None:
            print("Could not initialize correction system")
            return

        # Initialize speech components
        tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False)
        recognizer = initialize_enhanced_recognition()
        queue = []
        play_obj = None

        if not hasattr(whisper, 'cached_model'):
            whisper.cached_model = whisper.load_model("base")

        # Display patient information
        print("\nPatient Information Available:")
        print(f"Name: {patient['first_name']} {patient['last_name']}")
        print(f"DOB: {patient['date_of_birth']}")
        print(f"Member ID: {patient['member_number']}")
        print(f"Insurance: {patient['insurance_provider']}")
        print("\nStarting in Patient Information Phase\n")

        # Initial greeting
        initial_message = (
            f"Hi, this is an assistant from {office_name}. "
            f"I'm calling about our patient {patient['first_name']} {patient['last_name']}. "
            f"Would you mind helping me verify insurance coverage?"
        )
        initial_message = format_speech_output(initial_message)
        wav = np.array(tts.tts(text=initial_message))
        play_obj = handle_speech_output(queue, play_obj, wav, recognizer, None)

        # Main conversation loop
        while True:
            try:
                with sr.Microphone() as source:
                    # Handle interruptions during speech
                    if play_obj and play_obj.is_playing():
                        if recognizer.get_energy() > 3000:
                            play_obj.stop()
                            queue.clear()
                            audio = listen_for_speech(recognizer, source, play_obj)
                            if audio:
                                text = recognizer.recognize_whisper(audio, model="base")
                                print(f"Heard during speech: {text}")
                                continue

                    # Process queued audio
                    while len(queue) > 0:
                        if play_obj and play_obj.is_playing():
                            time.sleep(0.1)
                            continue
                        play_obj = queue.pop(0).play()

                    # Listen for speech
                    recognizer.adjust_for_ambient_noise(source, duration=1)
                    audio = listen_for_speech(recognizer, source, play_obj)
                    
                    if not audio:
                        continue

                    # Process speech input
                    text = recognizer.recognize_whisper(audio, model="base")
                    print(f"Original input: {text}")
                    
                    # Check for quit command
                    if text.lower() == "quit":
                        verification_summary = get_verification_summary(verification)
                        print("\nVerification Summary:")
                        for category, data in verification_summary['collected'].items():
                            if data:
                                print(f"\n{category.upper()}:")
                                for field, value in data.items():
                                    print(f"  {field}: {value}")
                        
                        with open('verification_results.json', 'w') as f:
                            json.dump(verification_summary, f, indent=2)
                        print("\nResults saved to verification_results.json")
                        break

                    # First check if input needs correction in context
                    validation_result = validate_input_context(text, verification, faiss_index, correction_df)
                    
                    if validation_result["needs_correction"]:
                        print(f"Input needs correction: {validation_result['reason']}")
                        correction_result = enhance_accent_handling(text, faiss_index, correction_df, verification)
                        
                        if correction_result['needs_confirmation']:
                            confirmation_msg = correction_result['confirmation_msg']
                            formatted_confirmation = format_speech_output(confirmation_msg)
                            print(f"Seeking confirmation: {formatted_confirmation}")
                            wav = np.array(tts.tts(text=formatted_confirmation))
                            play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
                            
                            confirmation_audio = listen_for_speech(recognizer, source, play_obj)
                            if confirmation_audio:
                                confirmation = recognizer.recognize_whisper(confirmation_audio, model="base")
                                if handle_confirmation(confirmation):
                                    text = correction_result['corrected']
                                    print(f"Confirmation received, using corrected input: {text}")
                                else:
                                    print("Correction rejected, asking for rephrasing")
                                    error_msg = "Could you please rephrase that?"
                                    wav = np.array(tts.tts(text=error_msg))
                                    play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
                                    continue
                        else:
                            text = correction_result['corrected']
                            print(f"Auto-corrected input: {text}")
                    else:
                        print("Input was valid, proceeding without correction")

                    # Process response and get next action
                    print(f"Processing response: {text}")
                    response, is_transition = flow_manager.process_response(text, verification)
                    print(f"Response: {response}, Is transition: {is_transition}")
                    
                    if response:
                        formatted_response = format_speech_output(response)
                        print(f"\nResponding: {formatted_response}")
                        wav = np.array(tts.tts(text=formatted_response))
                        play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)

                    # Check for verification completion
                    verification_summary = get_verification_summary(verification)
                    if verification_summary['status'] == 'complete':
                        print("\nVerification Complete!")
                        print("\nFinal Verification Summary:")
                        for category, data in verification_summary['collected'].items():
                            if data:
                                print(f"\n{category.upper()}:")
                                for field, value in data.items():
                                    print(f"  {field}: {value}")
                        
                        completion_message = "All verification information has been collected. Thank you for your help."
                        wav = np.array(tts.tts(text=completion_message))
                        play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
                        break

            except sr.UnknownValueError:
                print("Could not understand audio input")
                error_msg = "I'm sorry, I couldn't understand that. Could you please repeat?"
                wav = np.array(tts.tts(text=error_msg))
                play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
                
            except Exception as e:
                print(f"Error in main loop: {str(e)}")
                error_msg = "I encountered an error. Could you please rephrase that?"
                wav = np.array(tts.tts(text=error_msg))
                play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)

    finally:
        if play_obj and play_obj.is_playing():
            play_obj.stop()
        if hasattr(whisper, 'cached_model'):
            del whisper.cached_model

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        print("\nExiting gracefully...")
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
    finally:
        print("\nSession ended")

load INSTRUCTOR_Transformer
max_seq_length  512
Correction system initialized successfully
 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024

## Separating Code into Independent Components

Here, we outline the steps to separate the code into independent components and compile them together as `main.py`. Follow these instructions to run the script using `python main.py` from your terminal. I have put the debugging prints as it is for alpha testing.

### Steps

1. **Create Independent Components:**
   - Separate your functionalities into different modules (e.g., `utils.py`, `llm.py`, `stt.py`, `tts.py`, `verification.py`, `flow.py`).

2. **Combine Components in `main.py`:**
   - Import the necessary modules and libraries in `main.py`.
   - Integrate the functionalities in the `main()` function to manage the overall workflow.

3. **Setting Up the Virtual Environment:**
   - If you are downloading this from GitHub, please activate the virtual environment first depending on your system.

### Example Structure

1. **Directory Structure:**
   ```plaintext
   .
   ├── utils.py
   ├── llm.py
   ├── stt.py
   ├── tts.py
   ├── verification.py
   ├── flow.py
   ├── main.py
   ├── requirements.txt
   └── myenv/  # Virtual environment


In [1]:
import os
import numpy as np
import json
import warnings
import nltk
import time
import speech_recognition as sr
import whisper  # Added this import

from utils import (
    fake_patient, 
    format_speech_output, 
    get_verification_summary, 
    initialize_correction_system,
    validate_input_context,
    enhance_accent_handling,
    handle_confirmation
)
from llm import initialize_llm
from stt import initialize_enhanced_recognition, listen_for_speech
from tts import initialize_tts, handle_speech_output
from verification import InsuranceVerification
from flow import ConversationFlowManager, verify_patient

def main():
   current_play_obj = None  # Initialize at the start
   try:
       nltk.download('punkt_tab', quiet=True)
       warnings.filterwarnings('ignore')
       office_name = "Everest Dental Clinic"
       patient = fake_patient()
       verification = InsuranceVerification(office_name, patient)
       flow_manager = ConversationFlowManager(verification, patient)
       faiss_index, correction_df = initialize_correction_system()
       
       if faiss_index is None or correction_df is None:
           print("Could not initialize correction system")
           return

       tts = initialize_tts()
       recognizer = initialize_enhanced_recognition()
       queue = []
       play_obj = None

       if not hasattr(whisper, 'cached_model'):
           whisper.cached_model = whisper.load_model("base")

       # Display patient information
       print("\nPatient Information Available:")
       print(f"Name: {patient['first_name']} {patient['last_name']}")
       print(f"DOB: {patient['date_of_birth']}")
       print(f"Member ID: {patient['member_number']}")
       print(f"Insurance: {patient['insurance_provider']}")
       print("\nStarting in Patient Information Phase\n")

       # Initial greeting
       initial_message = (
           f"Hi, this is an assistant from {office_name}. "
           f"I'm calling about our patient {patient['first_name']} {patient['last_name']}. "
           f"Would you mind helping me verify insurance coverage?"
       )
       initial_message = format_speech_output(initial_message)
       wav = np.array(tts.tts(text=initial_message))
       current_play_obj = handle_speech_output(queue, play_obj, wav, recognizer, None)
       play_obj = current_play_obj

       # Main conversation loop
       while True:
           try:
               with sr.Microphone() as source:
                   # Handle interruptions during speech
                   if current_play_obj and current_play_obj.is_playing():
                       if recognizer.get_energy() > 3000:
                           current_play_obj.stop()
                           queue.clear()
                           audio = listen_for_speech(recognizer, source, current_play_obj)
                           if audio:
                               text = recognizer.recognize_whisper(audio, model="base")
                               print(f"Heard during speech: {text}")
                               continue

                   # Process queued audio
                   while len(queue) > 0:
                       if current_play_obj and current_play_obj.is_playing():
                           time.sleep(0.1)
                           continue
                       current_play_obj = queue.pop(0).play()
                       play_obj = current_play_obj

                   # Listen for speech
                   recognizer.adjust_for_ambient_noise(source, duration=1)
                   audio = listen_for_speech(recognizer, source, current_play_obj)
                   
                   if not audio:
                       continue

                   # Process speech input
                   text = recognizer.recognize_whisper(audio, model="base")
                   print(f"Original input: {text}")
                   
                   # Check for quit command
                   if text.lower() == "quit":
                       verification_summary = get_verification_summary(verification)
                       print("\nVerification Summary:")
                       for category, data in verification_summary['collected'].items():
                           if data:
                               print(f"\n{category.upper()}:")
                               for field, value in data.items():
                                   print(f"  {field}: {value}")
                       
                       with open('verification_results.json', 'w') as f:
                           json.dump(verification_summary, f, indent=2)
                       print("\nResults saved to verification_results.json")
                       break

                   # First check if input needs correction in context
                   validation_result = validate_input_context(text, verification, faiss_index, correction_df)
                   
                   if "confidence_too_low" in validation_result:
                       print(f"Low confidence: {validation_result['reason']}")
                       error_msg = "I didn't quite catch that. Could you please repeat?"
                       wav = np.array(tts.tts(text=error_msg))
                       current_play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
                       play_obj = current_play_obj
                       continue
                       
                   if validation_result["needs_correction"]:
                       print(f"Input needs correction: {validation_result['reason']}")
                       current_context = "insurance verification" if flow_manager.verification_started else "patient information"
                       correction_result = enhance_accent_handling(text, faiss_index, correction_df, verification, current_context)
                       
                       if correction_result['needs_confirmation']:
                           confirmation_msg = correction_result['confirmation_msg']
                           formatted_confirmation = format_speech_output(confirmation_msg)
                           print(f"Seeking confirmation: {formatted_confirmation}")
                           wav = np.array(tts.tts(text=formatted_confirmation))
                           current_play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
                           play_obj = current_play_obj
                           
                           confirmation_audio = listen_for_speech(recognizer, source, current_play_obj)
                           if confirmation_audio:
                               confirmation = recognizer.recognize_whisper(confirmation_audio, model="base")
                               if handle_confirmation(confirmation):
                                   text = correction_result['corrected']
                                   print(f"Confirmation received, using corrected input: {text}")
                               else:
                                   print("Correction rejected, asking for rephrasing")
                                   error_msg = "Could you please rephrase that?"
                                   wav = np.array(tts.tts(text=error_msg))
                                   current_play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
                                   play_obj = current_play_obj
                                   continue
                       else:
                           text = correction_result['corrected']
                           print(f"Auto-corrected input: {text}")
                   else:
                       print(f"Input valid: {validation_result['reason']}")

                   # Process response and get next action
                   print(f"Processing response: {text}")
                   response, is_transition = flow_manager.process_response(text, verification)
                   print(f"Response: {response}, Is transition: {is_transition}")
                   
                   if response:
                       formatted_response = format_speech_output(response)
                       print(f"\nResponding: {formatted_response}")
                       wav = np.array(tts.tts(text=formatted_response))
                       current_play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
                       play_obj = current_play_obj

                   # Check for verification completion
                   verification_summary = get_verification_summary(verification)
                   if verification_summary['status'] == 'complete':
                       print("\nVerification Complete!")
                       print("\nFinal Verification Summary:")
                       for category, data in verification_summary['collected'].items():
                           if data:
                               print(f"\n{category.upper()}:")
                               for field, value in data.items():
                                   print(f"  {field}: {value}")
                       
                       completion_message = "All verification information has been collected. Thank you for your help."
                       wav = np.array(tts.tts(text=completion_message))
                       current_play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
                       play_obj = current_play_obj
                       break

           except sr.UnknownValueError:
               print("Could not understand audio input")
               error_msg = "I'm sorry, I couldn't understand that. Could you please repeat?"
               wav = np.array(tts.tts(text=error_msg))
               current_play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
               play_obj = current_play_obj
               
           except Exception as e:
               print(f"Error in main loop: {str(e)}")
               error_msg = "I encountered an error. Could you please rephrase that?"
               wav = np.array(tts.tts(text=error_msg))
               current_play_obj = handle_speech_output(queue, play_obj, wav, recognizer, source)
               play_obj = current_play_obj

   finally:
       if current_play_obj and current_play_obj.is_playing():
           current_play_obj.stop()
       if hasattr(whisper, 'cached_model'):
           del whisper.cached_model

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        print("\nExiting gracefully...")
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
    finally:
        print("\nSession ended")

load INSTRUCTOR_Transformer
max_seq_length  512
Correction system initialized successfully
 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024